# Fine-Tuning MiniProject

**Name:** Luke Johnson  
**Date:** 1/15/2025  

## Goal
Fine-tune a pre-trained model for text classification. I'll use DistilBERT and create a simple dataset to work with.

## 1. Imports and Setup

In [1]:
# imports
import os
import json
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# pytorch stuff
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding
)

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# set seeds
np.random.seed(42)
torch.manual_seed(42)

warnings.filterwarnings('ignore')

print("imports done")
print(f"pytorch version: {torch.__version__}")

## 2. Create Dataset

In [2]:
# making a simple dataset for sentiment analysis
# 0 = negative, 1 = positive
np.random.seed(42)

texts = [
    "This is a positive example of good content.",
    "I really enjoyed this experience, it was wonderful.",
    "This product exceeded my expectations.",
    "I'm not satisfied with this service.",
    "This was a terrible experience, very disappointing.",
    "The quality is poor and I wouldn't recommend it.",
    "Amazing results, highly recommend!",
    "Outstanding performance and great value.",
    "Below average quality, needs improvement.",
    "Excellent customer service and fast delivery."
] * 100  # repeat to get more data

labels = [1, 1, 1, 0, 0, 0, 1, 1, 0, 1] * 100

df = pd.DataFrame({
    'text': texts,
    'label': labels
})

print(f"dataset shape: {df.shape}")
print(f"label counts:\n{df['label'].value_counts()}")
df.head()

In [3]:
# look at the data
print("dataset info:")
print(f"total samples: {len(df)}")

text_lengths = df['text'].str.len()
print(f"text length - mean: {text_lengths.mean():.1f}, std: {text_lengths.std():.1f}")

# plot label distribution
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
df['label'].value_counts().plot(kind='bar')
plt.title('Label Distribution')
plt.xlabel('Label')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
plt.hist(text_lengths, bins=20, alpha=0.7)
plt.title('Text Lengths')
plt.xlabel('Length')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# show some examples
print("\nsample texts:")
for i, (text, label) in enumerate(zip(df['text'][:3], df['label'][:3])):
    sentiment = "Positive" if label == 1 else "Negative"
    print(f"{i+1}. [{sentiment}] {text[:60]}...")

## 3. Split Data

In [4]:
# split into train/val/test
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['text'].tolist(), 
    df['label'].tolist(), 
    test_size=0.3, 
    random_state=42, 
    stratify=df['label']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, 
    temp_labels, 
    test_size=0.5, 
    random_state=42, 
    stratify=temp_labels
)

print(f"train: {len(train_texts)}, val: {len(val_texts)}, test: {len(test_texts)}")
print(f"train labels: {np.bincount(train_labels)}")
print(f"val labels: {np.bincount(val_labels)}")
print(f"test labels: {np.bincount(test_labels)}")

In [5]:
# setup tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"using: {model_name}")
print(f"vocab size: {tokenizer.vocab_size}")

# test it
sample_text = "This is a test."
tokens = tokenizer(sample_text, return_tensors="pt")
print(f"\nsample: {sample_text}")
print(f"token ids: {tokens['input_ids'][0][:5].tolist()}")

In [6]:
# dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# create datasets
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(val_texts, val_labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer)

print(f"datasets created!")
print(f"train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")

# test one
sample = train_dataset[0]
print(f"\nsample item - input shape: {sample['input_ids'].shape}, label: {sample['labels']}")

## 4. Load Model

In [7]:
# load the model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # binary classification
    ignore_mismatched_sizes=True
)

print(f"model loaded: {model_name}")
print(f"total params: {sum(p.numel() for p in model.parameters()):,}")
print(f"trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

print(f"\nconfig:")
print(f"  hidden size: {model.config.hidden_size}")
print(f"  layers: {model.config.num_hidden_layers}")
print(f"  labels: {model.config.num_labels}")

In [8]:
# training args
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    warmup_steps=100,
    report_to=None,
    seed=42
)

print("training args set:")
print(f"  lr: {training_args.learning_rate}")
print(f"  batch size: {training_args.per_device_train_batch_size}")
print(f"  epochs: {training_args.num_train_epochs}")

## 5. Train Model

In [9]:
# setup trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

print("trainer ready!")
print(f"train size: {len(train_dataset)}")
print(f"val size: {len(val_dataset)}")

In [10]:
# start training
print("training...")
print("-" * 30)

train_result = trainer.train()

print("-" * 30)
print("done!")
print(f"time: {train_result.metrics['train_runtime']:.1f}s")
print(f"loss: {train_result.metrics['train_loss']:.3f}")

In [11]:
# check validation
print("evaluating...")
eval_result = trainer.evaluate()

print("\nvalidation results:")
print(f"  loss: {eval_result['eval_loss']:.3f}")
print(f"  accuracy: {eval_result['eval_accuracy']:.3f}")
print(f"  time: {eval_result['eval_runtime']:.1f}s")

## 6. Test Model

In [12]:
# predict on test set
print("making predictions...")
test_predictions = trainer.predict(test_dataset)

# get predictions
predicted_labels = np.argmax(test_predictions.predictions, axis=1)
true_labels = test_predictions.label_ids

print(f"predictions done!")
print(f"pred shape: {predicted_labels.shape}")
print(f"true shape: {true_labels.shape}")

# show some examples
print("\nsample predictions:")
for i in range(min(3, len(predicted_labels))):
    pred = predicted_labels[i]
    true = true_labels[i]
    text = test_texts[i][:40] + "..." if len(test_texts[i]) > 40 else test_texts[i]
    print(f"  {text}")
    print(f"    true: {true}, pred: {pred}, {'right' if pred == true else 'wrong'}")
    print()

In [13]:
# calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
classification_rep = classification_report(true_labels, predicted_labels, target_names=['Negative', 'Positive'])
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("test results:")
print(f"accuracy: {accuracy:.3f}")
print(f"\nclassification report:")
print(classification_rep)

# plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negative', 'Positive'], 
            yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# more metrics
tn, fp, fn, tp = conf_matrix.ravel()
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"\nprecision: {precision:.3f}")
print(f"recall: {recall:.3f}")
print(f"f1: {f1:.3f}")

## 7. Look at Confidence

In [14]:
# check prediction confidence
prediction_probs = torch.softmax(torch.tensor(test_predictions.predictions), dim=1)
confidence_scores = torch.max(prediction_probs, dim=1)[0].numpy()

print("confidence analysis:")
print(f"mean: {confidence_scores.mean():.3f}")
print(f"std: {confidence_scores.std():.3f}")
print(f"min: {confidence_scores.min():.3f}")
print(f"max: {confidence_scores.max():.3f}")

# by correctness
correct = (predicted_labels == true_labels)
correct_conf = confidence_scores[correct]
wrong_conf = confidence_scores[~correct]

print(f"\ncorrect predictions - mean: {correct_conf.mean():.3f}")
print(f"wrong predictions - mean: {wrong_conf.mean():.3f}")

# plot
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(confidence_scores, bins=15, alpha=0.7)
plt.title('Confidence Distribution')
plt.xlabel('Confidence')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
plt.boxplot([correct_conf, wrong_conf], labels=['Correct', 'Wrong'])
plt.title('Confidence by Correctness')
plt.ylabel('Confidence')

plt.tight_layout()
plt.show()

In [15]:
# look at errors
print("error analysis:")
wrong_indices = np.where(predicted_labels != true_labels)[0]
print(f"wrong predictions: {len(wrong_indices)}")

if len(wrong_indices) > 0:
    print("\nsome wrong ones:")
    for i, idx in enumerate(wrong_indices[:3]):
        text = test_texts[idx]
        true_label = true_labels[idx]
        pred_label = predicted_labels[idx]
        conf = confidence_scores[idx]
        
        print(f"\n{i+1}. {text}")
        print(f"   true: {'Positive' if true_label == 1 else 'Negative'}")
        print(f"   pred: {'Positive' if pred_label == 1 else 'Negative'}")
        print(f"   conf: {conf:.3f}")
        
        # show tokens
        tokens = tokenizer(text, return_tensors='pt')
        print(f"   tokens: {tokenizer.convert_ids_to_tokens(tokens['input_ids'][0][:8])}")
else:
    print("no errors to look at!")

## 8. Save Model

In [16]:
# save everything
model_save_path = "./fine_tuned_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"model saved to: {model_save_path}")

# check what was saved
if os.path.exists(model_save_path):
    files = os.listdir(model_save_path)
    print(f"\nsaved files:")
    for file in files:
        if os.path.isfile(os.path.join(model_save_path, file)):
            size = os.path.getsize(os.path.join(model_save_path, file)) / (1024 * 1024)
            print(f"  {file}: {size:.1f} MB")

# save results
results = {
    'model_name': model_name,
    'train_metrics': train_result.metrics,
    'eval_metrics': eval_result,
    'test_metrics': {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    },
    'sizes': {
        'train': len(train_dataset),
        'val': len(val_dataset),
        'test': len(test_dataset)
    }
}

with open('./results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print(f"\nresults saved to: ./results.json")

In [17]:
# test loading
print("testing model loading...")

loaded_model = AutoModelForSequenceClassification.from_pretrained(model_save_path)
loaded_tokenizer = AutoTokenizer.from_pretrained(model_save_path)

print("loaded successfully!")

# test inference
test_text = "This is a test sentence."
inputs = loaded_tokenizer(test_text, return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    outputs = loaded_model(**inputs)
    predictions = torch.softmax(outputs.logits, dim=1)
    pred_class = torch.argmax(predictions, dim=1).item()
    confidence = torch.max(predictions, dim=1)[0].item()

print(f"\ntest: {test_text}")
print(f"prediction: {'Positive' if pred_class == 1 else 'Negative'}")
print(f"confidence: {confidence:.3f}")

## 9. Summary

In [18]:
# final summary
print("=== PROJECT SUMMARY ===")
print(f"\nmodel: {model_name}")
print(f"task: binary text classification")
print(f"dataset: {len(df)} samples")
print(f"splits: train={len(train_dataset)}, val={len(val_dataset)}, test={len(test_dataset)}")

print(f"\ntraining:")
print(f"  epochs: {training_args.num_train_epochs}")
print(f"  lr: {training_args.learning_rate}")
print(f"  batch size: {training_args.per_device_train_batch_size}")

print(f"\nresults:")
print(f"  train loss: {train_result.metrics['train_loss']:.3f}")
print(f"  val loss: {eval_result['eval_loss']:.3f}")
print(f"  test accuracy: {accuracy:.3f}")
print(f"  test f1: {f1:.3f}")

print(f"\nmodel saved to: {model_save_path}")
print(f"results saved to: ./results.json")

print("\nproject complete!")